**Ejercicio 2: Reforzamiento de las seguridades utilizando algoritmos criptográficos híbridos.**

Para reforzar la seguridad de las contraseñas en una organización y mitigar posibles riesgos de un ataque de fuerza bruta, se recomienda utilizar:

*Algoritmos asimétricos (por ejemplo, RSA o ECC)*, usado para proteger la transmisión de claves simétricas o elementos críticos.

*Algoritmos simétricos*, usado en el cifrado de datos de forma rápida.

*Hash con salt & pepper*, para fortalecer la seguridad de los Hashes mediante la adición de valores aleatorios (salt) y secretos (pepper).

Fabián Balseca Chávez.

**Instalación de librerías y dependencias.**

In [ ]:
import hashlib
import os
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.backends import default_backend

**Generación de las claves: privada y pública, con el algortimo RSA para proteger el salt & pepper.**

In [ ]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
    backend=default_backend()
)

public_key = private_key.public_key()


# Serialización de las claves para almacenamiento seguro (en aplicaciones reales)
pem_private = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
)

pem_public = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

**Sal & pepper generados de forma aleatoria para la protección de los datos.**

In [ ]:
salt = os.urandom(16)
pepper = os.urandom(16)

**Función para proteger la contraseña con algoritmos asimétricos RSA, Hash SHA-256, salt & pepper.**

In [ ]:
def secure_hash_with_rsa(password, salt, pepper, public_key):
    # Combina la contraseña con salt & pepper
    salted_password = salt + password.encode() + pepper

    # Cifrado de la contraseña con Hash SHA-256
    hashed_password = hashlib.sha256(salted_password).hexdigest()

    # Cifrado usando sal & pepper usando la clave pública
    encrypted_salt = public_key.encrypt(
        salt,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    encrypted_pepper = public_key.encrypt(
        pepper,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    # Devuelve el Hash y los elementos cifrados
    return hashed_password, encrypted_salt, encrypted_pepper

**Ejecución y prueba para verificar el aseguramiento de la contraseña.**

In [ ]:
password = "4a^T@6(^yhy7"
hashed_password, enc_salt, enc_pepper = secure_hash_with_rsa(password, salt, pepper, public_key)

print("\nContraseña:", password)
print("\nHash:", hashed_password)
print("\nSalt cifrado:", enc_salt)
print("\nPepper cifrado:", enc_pepper)
print("\nClave pública:", pem_public.decode())



Contraseña: 4a^T@6(^yhy7

Hash: 2be61861ab41a1bb34f02665746d87d6914830e957107fde0e0c248a9beb93d9

Salt cifrado: b'hx\x82\x02\xdcH\xfe\x1e\x9b\x06\xad\xb7w\x93\xc9\xdbr\x1b\x1e\xe5U\x14\xb9\xcbG\xe2g\xd4s\xfd\xf9\xc0#Z\x87\xc6\x80\xeb\x18\x83\x03\xdf\xec\x9d\xb5z\xb9\x9dk= "\'z\x8a\xd4\xc3\xe2\x8b\xa4\xa10b\xb1\xf7\xd0\xb0\r;l\x18\x08\xa67\xe9\xb7\xc1B:cl\xe6d\x81\x1dj\xa3\x91S\x1d\x10\x19\x15\xd8\x98\x89\xba\x0br\x00B\n*\xe3\xd1\xc7<\x18\'\xe3]\xd9\xd0g\x9a\x11{o\xbcn\xe7G6\x9e\xf3c\xe44\x8a\xc3s\x14-\x17\xda\x87\\\xee\xa7\xfaQus\x85z\\\xbd]\xaa \x93\xcf=\xae\xa6\xf4\xb6\n\xb3\x05\x0eP\xcd\x10"k\x13\xd1o\xa3d(\x11oe\x87\xbe\xd5\xdfk\xa5^\xac\xe6\x8f\xfa\xc4C\xb3B\x9c0\xa3\xceV\xe2s\x8d)\xf0\\Nf\xdc\x88\xfc\xe3j\x83\xcbK\xf5\x0f\xc5\xc0cM\xfeh.\xe5YPS\x84\x9b\x03\x9f.9E\x94M\xef7\x85HRZt2\xbc\xf7\xd0\xf48\x1b\x04xv\xf8U`/\xffL'

Pepper cifrado: b'\xd9\x0bbh\x1c\xf1\xa5\xb6o\xe7\x80\x1d\xa1\x00\xbev\x06WS\x87\x8d\xc2n\xca\x03\xb0\xf1\xcc!\xfd\xc2\xb6Pm\xdd\x04\x94\xbc\xd2oQ\xc3c\x87,C\x